In [1]:
import numpy as np
from datetime import datetime
import time
from siriuspy.search import IDSearch
from apsuite.commisslib.meas_bpms_signals import AcqBPMsSignals
from siriuspy.devices import BOPSRampStandbyHandler, BORFRampStandbyHandler, \
    BunchbyBunch, HLFOFB, SOFB, IVU


In [2]:
def initialize_ivu(beamline):
    # Search ID
    devnameivu = IDSearch.conv_beamline_2_idname(beamline=beamline)
    ivu = IVU(devname=devnameivu)

    # Disable beamline control
    ivu.cmd_beamline_ctrl_disable()
    print('beamline control: ', ivu.is_beamline_ctrl_enabled)

    # Disable center mode and pitch mode
    ivu.set_center_mode(False)
    ivu.set_pitch_mode(False)
    time.sleep(0.5)
    print('center mode: ', ivu.center_mode_status)
    print('pitch mode: ', ivu.pitch_mode_status)

    # Set gap speed
    ivu.set_gap_speed(0.1)
    time.sleep(0.5)
    print('gap speed: {:.3f} mm/s'.format(ivu.gap_speed))

    # Set gap to parked condition
    ivu.set_gap(ivu.kparameter_parked)
    time.sleep(0.5)
    print('gap: {:.3f} mm'.format(ivu.gap))

    return ivu

def move_ivu_gap(ivu:IVU, gap, timeout, verbose=False):
    ivu.set_gap(gap)
    time.sleep(0.5)
    print('Gap-RB {:.3f} mm'.format(ivu.gap)) if verbose else 0
    if ivu.cmd_move_gap_start(timeout):
        time.sleep(0.5)
        print('Undulator is moving...') if verbose else 0
        while ivu.is_moving:
            time.sleep(0.1)
            print('Current gap {:.3f} mm.'.format(ivu.gap_mon), end='\r') if verbose else 0
        print('Gap {:.3f} mm reached.'.format(ivu.gap)) if verbose else 0
        return True
    else:
        print('Error while cmd_move_start.')
        return False
    
def configure_acquisition_params(orbacq, timeout=40, nrptsbefore=1000, nrptsafter=10000):
    """."""
    params = orbacq.params
    params.signals2acq = 'XY'
    params.acq_rate = 'FAcq' 
    params.timeout = 40

    params.nrpoints_before = nrptsbefore  # Default: 1000
    params.nrpoints_after = nrptsafter  # Default: 10000
    params.acq_repeat = False
    params.trigbpm_delay = None
    params.trigbpm_nrpulses = 1

    params.timing_event = 'Study' 
    params.event_mode = 'External'
    params.event_delay = None
    params.do_pulse_evg = False  # Default: False
    print('--- orbit acquisition configuration ---')
    print(params)


def acquire_data_moving(orbacq, ivu, start_gap, end_gap):
    """."""
    fambpms = orbacq.devices["fambpms"]
    ret = orbacq.prepare_bpms_acquisition()
    tag = orbacq._bpm_tag(idx=abs(int(ret))-1)
    if ret < 0:
        print(tag + " did not finish last acquisition.")
    elif ret > 0:
        print(tag + " is not ready for acquisition.")

    fambpms.reset_mturn_initial_state()
    orbacq.trigger_timing_signal()
    evt = orbacq._get_event(orbacq.params.timing_event)

    move_ivu_gap(ivu, gap=start_gap, timeout=3, verbose=True)
    ivu.set_gap(end_gap)
    time.sleep(0.5)
    print('Gap-RB {:.3f} mm'.format(ivu.gap))
    ivu.cmd_move_gap_start(timeout=3)
    evt.cmd_external_trigger()

    time0 = time.time()
    ret = fambpms.wait_update_mturn(timeout=orbacq.params.timeout)
    print(f"it took {time.time()-time0:02f}s to update bpms")
    if ret != 0:
        print("There was a problem with acquisition")
        if ret > 0:
            tag = orbacq._bpm_tag(idx=int(ret)-1)
            pos = fambpms.mturn_signals2acq[int((ret % 1) * 10) - 1]
            print("This BPM did not update: " + tag + ", signal " + pos)
        elif ret == -1:
            print("Initial timestamps were not defined")
        elif ret == -2:
            print("Signals size changed.")
        return
    orbacq.data = orbacq.get_data()
    

def measure(orbacq, ivu, start_gap, end_gap):
    """."""
    init_state = orbacq.get_timing_state()
    orbacq.prepare_timing()
    print((
        'Please check BPM acquisition status, or wait a few seconds, '
        ' before triggering event!'))
    try:
        acquire_data_moving(orbacq, ivu, start_gap, end_gap)
    except Exception as e:
        print(f"An error occurred during acquisition: {e}")
    # Restore initial timing state, regardless acquisition status
    orbacq.recover_timing_state(init_state)
    return orbacq.data is not None


def initialize(timeout, nrptsbefore, nrptsafter):
    """."""
    orbacq = AcqBPMsSignals(isonline=True)
    configure_acquisition_params(orbacq, timeout, nrptsbefore, nrptsafter)
    print('--- orbit acquisition connection ---')
    print(orbacq.wait_for_connection(timeout=100))
    return orbacq


def create_devices():
    """."""
    sofb = SOFB(SOFB.DEVICES.SI)
    fofb = HLFOFB(HLFOFB.DEVICES.SI)
    bbbl = BunchbyBunch(BunchbyBunch.DEVICES.L)
    bbbh = BunchbyBunch(BunchbyBunch.DEVICES.H)
    bbbv = BunchbyBunch(BunchbyBunch.DEVICES.V)
    bopsrmp = BOPSRampStandbyHandler()
    borfrmp = BORFRampStandbyHandler()
    return sofb, fofb, bbbl, bbbh, bbbv, bopsrmp, borfrmp


def read_feedback_status(devs, orbacq):
    """."""
    sofb, fofb, bbbl, bbbh, bbbv, bopsrmp, borfrmp = devs
    orbacq.data['sofb_loop_state'] = sofb.autocorrsts
    orbacq.data['fofb_loop_state'] = fofb.loop_state
    orbacq.data['bbbl_loop_state'] = bbbl.feedback.loop_state
    orbacq.data['bbbh_loop_state'] = bbbh.feedback.loop_state
    orbacq.data['bbbv_loop_state'] = bbbv.feedback.loop_state
    orbacq.data['bo_ps_ramp_state'] = bopsrmp.is_on
    orbacq.data['bo_rf_ramp_state'] = borfrmp.is_on

In [ ]:
timeout = 40
nrptsbefore=1000
nrptsafter=10000

start_gap = 24
end_gap = 20
gap_speed = 0.5
beamline='EMA'


ivu = initialize_ivu(beamline=beamline)
ivu.set_gap_speed(gap_speed)

filename_prefix = 'IVU18_' + beamline + '_{:.1f}-{:.1f}_mm'.format(start_gap, end_gap)
filename_prefix += '_gspeed_{:.3f}'.format(gap_speed)

orbacq = initialize(timeout, nrptsbefore, nrptsafter)
devs = create_devices()

if measure(orbacq, ivu, start_gap, end_gap):
    read_feedback_status(devs, orbacq)
    now = datetime.now()
    str_rate = f'FAcq_rate_'
    str_now = now.strftime('%Y-%m-%d_%Hh%Mm%Ss')
    filename = filename_prefix + str_rate + str_now
    orbacq.save_data(filename, overwrite=False)
    print(f'\nData saved at {str_now:s}')
else:
    print('\nData NOT saved!')